In [1]:
!python -m spacy validate
!python -m spacy download ru_core_news_sm


| Loading compatibility table...
/ Loading compatibility table...
- Loading compatibility table...
\ Loading compatibility table...
[+] Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.2) =================
[i] spaCy installation:
C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\spacy

NAME              SPACY            VERSION                              
en_core_web_sm    >=3.8.0,<3.9.0   3.8.0   [+]
ru_core_news_sm   >=3.8.0,<3.9.0   3.8.0   [+]

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
      --------------------------------------- 0.3/15.3 MB ? eta -:--:--
     - -------------------------------------- 0.5/15.3 MB 1.3 MB/s eta 0:00:12
     -- ------------------------------------- 0.8/15.3 MB 1.2 MB/s eta 0:00:12
     -- ------------------------------------- 0.8/15

In [3]:
# Импорт необходимых библиотек
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy

# Загрузка модели для нормализации и токенизации текста
nlp = spacy.blank("ru")  # Загружаем пустую модель spaCy для русского языка

# Функция нормализации и токенизации
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clean_text(text) for text in X]

    def clean_text(self, text):
        # Убираем нежелательные символы
        text = re.sub(r'[^\w\s]', '', text.lower())
        # Токенизация и лемматизация
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
        return " ".join(tokens)

# Загрузка данных SIB200
def load_sib200_ru():
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if unknown_categories:
        raise RuntimeError(f'Категории {unknown_categories} присутствуют в валидации, но отсутствуют в обучении.')
    unknown_categories = set(y_test) - categories
    if unknown_categories:
        raise RuntimeError(f'Категории {unknown_categories} присутствуют в тесте, но отсутствуют в обучении.')
    
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

# Загружаем данные
(X_train, y_train), (X_val, y_val), (X_test, y_test), categories = load_sib200_ru()

# Полное разделение на обучение и валидацию
X_train_full = X_train + X_val
y_train_full = y_train + y_val

# Создаем Pipeline для предобработки текста, TF-IDF и классификаторов
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3), max_features=10000)),  # Униграммы, биграммы и триграммы
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42))
])

# Определяем сетку параметров для GridSearchCV
param_grid = [
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)],
        'classifier__C': [0.1, 1, 10]
    },
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [RandomForestClassifier(class_weight='balanced', random_state=42)],
        'classifier__n_estimators': [100, 200, 10],
        'classifier__max_depth': [10, 20, None]
    },
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [GradientBoostingClassifier(random_state=42)],
        'classifier__n_estimators': [100, 200, 10],
        'classifier__learning_rate': [0.01, 0.1, 0.5]
    }
]

# Настройка GridSearchCV с кросс-валидацией
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(5), scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train_full, y_train_full)

# Печать лучших параметров и наилучшего результата
print("Лучшие параметры:", grid_search.best_params_)
print("Наилучшее значение F1:", grid_search.best_score_)

# Оценка на валидационной и тестовой выборках
y_val_pred = grid_search.predict(X_val)
y_test_pred = grid_search.predict(X_test)

# Отчет классификации и матрица ошибок для валидационной выборки
print("Отчет классификации (валидация):\n", classification_report(y_val, y_val_pred, target_names=categories))
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_val, y_val_pred), annot=True, fmt="d", cmap="Blues", xticklabels=categories, yticklabels=categories)
plt.xlabel("Предсказанные метки")
plt.ylabel("Истинные метки")
plt.title("Матрица ошибок (валидация)")
plt.show()

# Отчет классификации и матрица ошибок для тестовой выборки
print("Отчет классификации (тест):\n", classification_report(y_test, y_test_pred, target_names=categories))
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="Blues", xticklabels=categories, yticklabels=categories)
plt.xlabel("Предсказанные метки")
plt.ylabel("Истинные метки")
plt.title("Матрица ошибок (тест)")
plt.show()

['Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным.',
 'В начале войны они в основном передвигались по поверхности моря, но когда радары начали совершенствоваться и становились более точными, подводным лодкам пришлось уйти под воду, чтобы их не могли обнаружить.',
 'По мере того как знание греческого языка ухудшалось, Запад оказался отрезанным от своих философских и научных греческих корней.',
 'Впрочем, зимой это другая красота и шарм, с горными деревушками, где выпадает много снега и доступны такие занятия, как катание на горные лыжах и сноуборде.',
 'В этих отелях останавливались богатые и известные люди тех времён, и часто у них было изысканное питание и ночная жизнь',
 'В 1994 году этот конфликт привёл к созданию на востоке Молдовы самопровозглашённой Приднестровской республики, которая имеет собственное правительство и валюту, но не признана ни одним государством-членом ООН.',
 'Даниэль Лантань, эксперт ООН по вопросам данной б

Fitting 5 folds for each of 126 candidates, totalling 630 fits


ValueError: 
All the 630 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
630 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy

# Загрузка модели для нормализации и токенизации текста
nlp = spacy.load("ru_core_news_sm")  # Загружаем модель spaCy для русского языка

# Функция нормализации и токенизации
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clean_text(text) for text in X]

    def clean_text(self, text):
        # Убираем нежелательные символы
        text = re.sub(r'[^\w\s]', '', text.lower())
        # Токенизация и лемматизация
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        return " ".join(tokens)

# Загрузка данных SIB200
def load_sib200_ru():
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if unknown_categories:
        raise RuntimeError(f'Категории {unknown_categories} присутствуют в валидации, но отсутствуют в обучении.')
    unknown_categories = set(y_test) - categories
    if unknown_categories:
        raise RuntimeError(f'Категории {unknown_categories} присутствуют в тесте, но отсутствуют в обучении.')
    
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

# Загружаем данные
(X_train, y_train), (X_val, y_val), (X_test, y_test), categories = load_sib200_ru()

# Полное разделение на обучение и валидацию
X_train_full = X_train + X_val
y_train_full = y_train + y_val

# Создаем Pipeline для предобработки текста, TF-IDF и классификаторов
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor(nlp)),
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3), max_features=10000)),  # Униграммы, биграммы и триграммы
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42))
])

# Определяем сетку параметров для GridSearchCV
param_grid = [
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)],
        'classifier__C': [0.1, 1, 10]
    },
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [RandomForestClassifier(class_weight='balanced', random_state=42)],
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20, None]
    },
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [GradientBoostingClassifier(random_state=42)],
        'classifier__n_estimators': [100, 200],
        'classifier__learning_rate': [0.01, 0.1, 0.5]
    }
]

# Настройка GridSearchCV с кросс-валидацией
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(5), scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train_full, y_train_full)

# Печать лучших параметров и наилучшего результата
print("Лучшие параметры:", grid_search.best_params_)
print("Наилучшее значение F1:", grid_search.best_score_)

# Оценка на валидационной и тестовой выборках
y_val_pred = grid_search.predict(X_val)
y_test_pred = grid_search.predict(X_test)

# Отчет классификации и матрица ошибок для валидационной выборки
print("Отчет классификации (валидация):\n", classification_report(y_val, y_val_pred, target_names=categories))
disp_val = ConfusionMatrixDisplay(confusion_matrix(y_val, y_val_pred), display_labels=categories)
disp_val.plot(cmap=plt.cm.Blues)
plt.title("Матрица ошибок (валидация)")
plt.show()

# Отчет классификации и матрица ошибок для тестовой выборки
print("Отчет классификации (тест):\n", classification_report(y_test, y_test_pred, target_names=categories))
disp_test = ConfusionMatrixDisplay(confusion_matrix(y_test, y_test_pred), display_labels=categories)
disp_test.plot(cmap=plt.cm.Blues)
plt.title("Матрица ошибок (тест)")
plt.show()

Fitting 5 folds for each of 90 candidates, totalling 450 fits


In [ ]:
# Импорт необходимых библиотек
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy

# Загрузка модели для нормализации и токенизации текста
nlp = spacy.load("ru_core_news_sm")  # Загружаем модель spaCy для русского языка

# Функция нормализации и токенизации
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.nlp = spacy.load("ru_core_news_sm")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clean_text(text) for text in X]

    def clean_text(self, text):
        # Убираем нежелательные символы
        text = re.sub(r'[^\w\s]', '', text.lower())
        # Токенизация и лемматизация
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        return " ".join(tokens)

# Загрузка данных SIB200
def load_sib200_ru():
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if unknown_categories:
        raise RuntimeError(f'Категории {unknown_categories} присутствуют в валидации, но отсутствуют в обучении.')
    unknown_categories = set(y_test) - categories
    if unknown_categories:
        raise RuntimeError(f'Категории {unknown_categories} присутствуют в тесте, но отсутствуют в обучении.')
    
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

# Загружаем данные
(X_train, y_train), (X_val, y_val), (X_test, y_test), categories = load_sib200_ru()

# Проверка первых строк данных
print("Первые строки тренировочных данных:")
print(pd.DataFrame({'text': X_train, 'label': y_train}).head())

# Полное разделение на обучение и валидацию
X_train_full = X_train + X_val
y_train_full = y_train + y_val

# Создаем Pipeline для предобработки текста, TF-IDF и классификаторов
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3), max_features=10000)),  # Униграммы, биграммы и триграммы
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42))
])

# Определяем сетку параметров для GridSearchCV
param_grid = [
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)],
        'classifier__C': [0.1, 1, 10]
    },
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [RandomForestClassifier(class_weight='balanced', random_state=42)],
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20, None]
    },
    {
        'tfidf__max_features': [5000, 10000, 15000],
        'tfidf__ngram_range': [(1, 2), (1, 3)],
        'classifier': [GradientBoostingClassifier(random_state=42)],
        'classifier__n_estimators': [100, 200],
        'classifier__learning_rate': [0.01, 0.1, 0.5]
    }
]

# Настройка GridSearchCV с кросс-валидацией
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(5), scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train_full, y_train_full)

# Печать лучших параметров и наилучшего результата
print("Лучшие параметры:", grid_search.best_params_)
print("Наилучшее значение F1:", grid_search.best_score_)

# Оценка на валидационной и тестовой выборках
y_val_pred = grid_search.predict(X_val)
y_test_pred = grid_search.predict(X_test)

# Отчет классификации и матрица ошибок для валидационной выборки
print("Отчет классификации (валидация):\n", classification_report(y_val, y_val_pred, target_names=categories))
disp_val = ConfusionMatrixDisplay(confusion_matrix(y_val, y_val_pred), display_labels=categories)
disp_val.plot(cmap=plt.cm.Blues)
plt.title("Матрица ошибок (валидация)")
plt.show()

# Отчет классификации и матрица ошибок для тестовой выборки
print("Отчет классификации (тест):\n", classification_report(y_test, y_test_pred, target_names=categories))
disp_test = ConfusionMatrixDisplay(confusion_matrix(y_test, y_test_pred), display_labels=categories)
disp_test.plot(cmap=plt.cm.Blues)
plt.title("Матрица ошибок (тест)")
plt.show()

# Визуализация ROC-кривых для каждой категории
def plot_roc_curve(y_true, y_pred_proba, categories):
    y_true_binarized = label_binarize(y_true, classes=list(range(len(categories))))
    n_classes = y_true_binarized.shape[1]
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    plt.figure(figsize=(12, 8))
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {categories[i]} (area = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()

# Получение вероятностей классов
y_test_pred_proba = grid_search.predict_proba(X_test)

# Построение ROC-кривых
plot_roc_curve(y_test, y_test_pred_proba, categories)

# Визуализация важности признаков для логистической регрессии
if isinstance(grid_search.best_estimator_.named_steps['classifier'], LogisticRegression):
    feature_importances = grid_search.best_estimator_.named_steps['classifier'].coef_
    feature_names = grid_search.best_estimator_.named_steps['tfidf'].get_feature_names_out()
    
    # Визуализация важности признаков для первой категории
    top_n = 20
    for i, category in enumerate(categories):
        top_indices = np.argsort(feature_importances[i])[-top_n:]
        top_features = [feature_names[idx] for idx in top_indices]
        top_importances = feature_importances[i][top_indices]
        
        plt.figure(figsize=(12, 6))
        sns.barplot(x=top_importances, y=top_features)
        plt.title(f'Важность признаков для категории {category}')
        plt.xlabel('Важность')
        plt.ylabel('Признаки')
        plt.show()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start(

AttributeError: _ARRAY_API not found

Первые строки тренировочных данных:
                                                text  label
0  Турция с трёх сторон окружена морями: на запад...      1
1  В начале войны они в основном передвигались по...      4
2  По мере того как знание греческого языка ухудш...      4
3  Впрочем, зимой это другая красота и шарм, с го...      0
4  В этих отелях останавливались богатые и извест...      0
Fitting 5 folds for each of 90 candidates, totalling 450 fits
